In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
import nltk.corpus

In [9]:
data=pd.read_csv("train.csv")
train_data=data.drop(["keyword","location"],axis=1)
train_data
type(train_data)

pandas.core.frame.DataFrame

In [7]:
test_data=pd.read_csv('test.csv')
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [8]:
test_data=test_data.drop(["keyword","location"],axis=1)
test_data

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,Storm in RI worse than last hurricane. My city...
3260,10868,Green Line derailment in Chicago http://t.co/U...
3261,10874,MEG issues Hazardous Weather Outlook (HWO) htt...


In [26]:
datatrain=pd.DataFrame(train_data.text,columns=["text"])
datatest=pd.DataFrame(test_data.text,columns=["text"])
data=pd.concat([datatrain,datatest],axis=0,ignore_index=True)
data


,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...
...,...
10871,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
10872,Storm in RI worse than last hurricane. My city...
10873,Green Line derailment in Chicago http://t.co/U...
10874,MEG issues Hazardous Weather Outlook (HWO) htt...


In [27]:
import re
import string

def function_for_handle_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def function_for_handle_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)



In [28]:
data['text']=data['text'].apply(lambda x:function_for_handle_url(x))


In [29]:
for i in range(len(data)):
    data["text"][i].lower()


In [40]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [42]:
import gensim
data["tokenized_text"]=data["text"].apply(gensim.utils.simple_preprocess)
data.head()

C:\Users\gopal\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


,text,tokenized_text
0,Our Deeds are the Reason of this #earthquake M...,"[our, deeds, are, the, reason, of, this, earth..."
1,Forest fire near La Ronge Sask. Canada,"[forest, fire, near, la, ronge, sask, canada]"
2,All residents asked to 'shelter in place' are ...,"[all, residents, asked, to, shelter, in, place..."
3,"13,000 people receive #wildfires evacuation or...","[people, receive, wildfires, evacuation, order..."
4,Just got sent this photo from Ruby #Alaska as ...,"[just, got, sent, this, photo, from, ruby, ala..."


In [44]:
def revove_stopwords(text):
    text_clean=[word for word in text if word not in stop_words]
    return text_clean
data['clean_tok_text']=data['tokenized_text'].apply(lambda x:revove_stopwords(x))
data.head()

,text,tokenized_text,clean_tok_text
0,Our Deeds are the Reason of this #earthquake M...,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv..."
1,Forest fire near La Ronge Sask. Canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]"
2,All residents asked to 'shelter in place' are ...,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o..."
3,"13,000 people receive #wildfires evacuation or...","[people, receive, wildfires, evacuation, order...","[people, receive, wildfires, evacuation, order..."
4,Just got sent this photo from Ruby #Alaska as ...,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi..."


In [66]:
def join_fun(text):
    string=" "
    ans=string.join(text)
    return ans
data['final_text']=data['clean_tok_text'].apply(lambda x:join_fun(x))
data.head()

,text,tokenized_text,clean_tok_text,final_text
0,Our Deeds are the Reason of this #earthquake M...,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",deeds reason earthquake may allah forgive us
1,Forest fire near La Ronge Sask. Canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",residents asked shelter place notified officer...
3,"13,000 people receive #wildfires evacuation or...","[people, receive, wildfires, evacuation, order...","[people, receive, wildfires, evacuation, order...",people receive wildfires evacuation orders cal...
4,Just got sent this photo from Ruby #Alaska as ...,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",got sent photo ruby alaska smoke wildfires pou...


In [67]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

vec=cv.fit_transform(data["final_text"])

In [68]:
X=vec[:len(train_data)]
y=train_data.target
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.90)

In [69]:
model=RandomForestClassifier(n_estimators=65)
model.fit(X_train,y_train)
pred=model.predict(X_test)


In [70]:
accuracy_score(pred,y_test)

0.7795275590551181

In [75]:
test_vac=vec[len(train_data):]
test_vac

<3263x20023 sparse matrix of type '<class 'numpy.int64'>'
	with 29200 stored elements in Compressed Sparse Row format>

In [82]:
predictios=model.predict(test_vac)
dataframe=pd.DataFrame(predictios,columns=["target"])
submission=pd.read_csv("sample_submission.csv")
submission['target']=dataframe
submission.to_csv('my_submission.csv',index=False)